In [1]:
import numpy as np

# ---------------------
# Helper functions
# ---------------------
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)


In [2]:
# Generator: z -> x_fake
class Generator:
    def __init__(self):
        # single linear layer: x_fake = w*z + b
        self.w = np.random.randn() * 0.1
        self.b = 0.0
    
    def forward(self, z):
        return self.w * z + self.b
    
    def params(self):
        return [self.w, self.b]
    
    def update(self, dw, db, lr=0.01):
        self.w -= lr * dw
        self.b -= lr * db


In [3]:
# Discriminator: x -> probability real
class Discriminator:
    def __init__(self):
        # single-layer logistic regression
        self.w = np.random.randn() * 0.1
        self.b = 0.0
    
    def forward(self, x):
        return sigmoid(self.w * x + self.b)
    
    def params(self):
        return [self.w, self.b]
    
    def update(self, dw, db, lr=0.01):
        self.w -= lr * dw
        self.b -= lr * db

In [4]:
# ---------------------
# Training loop
# ---------------------
np.random.seed(42)

G = Generator()
D = Discriminator()

lr = 0.01
num_steps = 20000
batch_size = 32

for step in range(num_steps):
    # 1. Sample real data from N(3,1)
    real_data = np.random.normal(3, 1, batch_size)
    
    # 2. Sample noise and generate fake data
    z = np.random.uniform(-1, 1, batch_size)
    fake_data = G.forward(z)
    
    # 3. Discriminator forward
    D_real = D.forward(real_data)
    D_fake = D.forward(fake_data)
    
    # 4. Compute losses
    loss_D = -np.mean(np.log(D_real + 1e-8) + np.log(1 - D_fake + 1e-8))
    loss_G = -np.mean(np.log(D_fake + 1e-8))
    
    # 5. Backprop for Discriminator
    dL_dDreal = -1 / (D_real + 1e-8)
    dL_dDfake = -1 / (1 - D_fake + 1e-8)
    
    dDreal_dout = D_real * (1 - D_real)
    dDfake_dout = D_fake * (1 - D_fake)
    
    grad_real = dL_dDreal * dDreal_dout
    grad_fake = dL_dDfake * (-dDfake_dout)
    
    dDw = np.mean(grad_real * real_data + grad_fake * fake_data)
    dDb = np.mean(grad_real + grad_fake)
    
    D.update(dDw, dDb, lr)
    
    # 6. Backprop for Generator (through D)
    dL_dDfake_G = -1 / (D_fake + 1e-8)  # gradient of G’s loss wrt D output
    grad_fake_G = dL_dDfake_G * dDfake_dout
    
    dG_fake = grad_fake_G * D.w   # propagate through D’s linear input
    dGw = np.mean(dG_fake * z)
    dGb = np.mean(dG_fake)
    
    G.update(dGw, dGb, lr)
    
    # 7. Logging
    if step % 2000 == 0:
        print(f"Step {step}: Loss_D={loss_D:.3f}, Loss_G={loss_G:.3f}, "
              f"Sample_fake_mean={np.mean(fake_data):.2f}")


Step 0: Loss_D=1.406, Loss_G=0.693, Sample_fake_mean=-0.00
Step 2000: Loss_D=1.371, Loss_G=0.884, Sample_fake_mean=4.25
Step 4000: Loss_D=1.394, Loss_G=0.604, Sample_fake_mean=2.43
Step 6000: Loss_D=1.389, Loss_G=0.741, Sample_fake_mean=3.28
Step 8000: Loss_D=1.386, Loss_G=0.680, Sample_fake_mean=2.87
Step 10000: Loss_D=1.387, Loss_G=0.710, Sample_fake_mean=3.06
Step 12000: Loss_D=1.387, Loss_G=0.681, Sample_fake_mean=2.98
Step 14000: Loss_D=1.386, Loss_G=0.695, Sample_fake_mean=3.01
Step 16000: Loss_D=1.386, Loss_G=0.697, Sample_fake_mean=3.00
Step 18000: Loss_D=1.386, Loss_G=0.700, Sample_fake_mean=3.00
